# Example

## Running cuOpt Server for CVRPTW

In [ ]:
import json
import requests

**Set-up the IP, Port where the server is running**

***

In this example, the server was running on localhost (ip: 0.0.0.0) port 5000.

In [ ]:
ip = "0.0.0.0"
port = "5000"

## Accumulate problem data

In [ ]:
json_data = {}


**Set waypoint graph**

***

Need to provide offsets, edges, and weights; weights can't be negative values.

Considering 2 types of vehicle, a Van and a Truck, Van would be faster/costs less in few paths compared to Trucks, which would depict change in weights of the cost of trvael

Similarly **travel_time_graph_data** can also be set.

In [ ]:
# Van
graph_data_1 = {
    "offsets": [0,       3,    5,           9,    11,   13,   15,   17, 18, 19, 20, 21],
    "edges":   [1, 2, 9, 0, 7, 0, 3, 4, 10, 2, 4, 2, 5, 6, 9, 5, 8, 1,  6,  0,  5],
    "weights": [1, 1, 2, 1, 2, 1, 1, 1,  3, 2, 3, 2, 1, 2, 1, 3, 4, 2,  3,  1,  1] 
}

# Truck
graph_data_2 = graph_data_1.copy()
graph_data_2["weights"] = [2, 2, 3, 2, 3, 2, 2, 2, 4, 4, 4, 3, 2, 3, 2, 4, 5, 3,  4,  2,  1] 

json_data["cost_waypoint_graph_data"] = {
    'waypoint_graph': {
        1: graph_data_1, # Van
        2: graph_data_2  # Truck
    }
}

json_data["travel_time_waypoint_graph_data"] = {
    'waypoint_graph': {
        1: graph_data_1, # Van
        2: graph_data_2  # Truck
    }
}

**Set Fleet data**

***

Provide vehicle start and end locations along with vehicle features and capacity.

In [ ]:
fleet_data = {
    "vehicle_locations": [[0, 0], [1, 1], [0, 1], [1, 0], [0, 0]],
    "vehicle_ids": ["Van-A", "Truck-A", "Van-B", "Truck-B", "Van-C"], 
    "vehicle_types": [1, 2, 1, 2, 1],
    "capacities": [[10, 12, 15, 8, 10]],
    "vehicle_time_windows": [
        [0, 80],
        [1, 40], 
        [3, 30], 
        [5, 80], 
        [20, 100]
    ],
    # Vehicle can take breaks in this time window as per berak duration provided
    "vehicle_break_time_windows":[
        [
            [20, 25],
            [20, 25], 
            [20, 25], 
            [20, 25], 
            [20, 25]
        ]
    ],
    "vehicle_break_durations": [[1, 1, 1, 1, 1]],
    # Vehicle Id 0 can only serve Order 0 and 4
    "vehicle_order_match": [
        {
            "vehicle_id": 0,
            "order_ids": [0, 4]
        }
    ],  # 0th vehicle can only serve 0th and 4th order only
    # Don't count trip from depot to first task location
    "skip_first_trips": [False, True, True, False, False], 
    # Don't count trip from last task location to depot
    "drop_return_trips": [False, True, True, False, False],
    # Maximum cost a vehicle can incur while delivering
    "vehicle_max_costs": [100, 100, 100, 100, 100],
    # Maximum time a vehicle can be working
    "vehicle_max_times": [120, 120, 120, 120, 120],
    # Minimum 2 vehicles are required to be in solution 
    "min_vehicles": 2,
}

json_data["fleet_data"] = fleet_data

**Set Task data**

***

Provide details on task locations, demand and time window, there are other options as well.

In [ ]:
task_data = {
    "task_locations": [1, 3, 4, 6, 8],
    "demand": [[3, 4, 4, 3, 2]],
    "task_time_windows": [
        [3, 20],
        [5, 30],
        [1, 20],
        [4, 40],
        [0, 30],
    ],
    "service_times": [3, 1, 8, 4, 0],
    "prizes": [10, 20, 20, 30, 5],
    # Order Id 0 and 4 can be served only by vehicle with id 0
    "order_vehicle_match": [
        {
            "order_id": 0,
            "vehicle_ids": [0]
        },
        {
            "order_id": 4,
            "vehicle_ids": [0]
        }
    ]
}

json_data["task_data"] = task_data

**Set solver config**

***

larger problems might require more time and/or more climbers

In [ ]:
solver_config = {
    "time_limit": 1
}

json_data["solver_config"] = solver_config

**Complete Problem data**

In [ ]:
print(json.dumps(json_data, indent=4))

**Solve the problem**

***

By default the solver will use all data provided and any constraints implied by that data. For example, if capacity data was provided within the vehicle data, it is assumed that the user desires to set a capacity constraint on the vehicles. If, however, a user would like to ignore particular aspects of the provided data at solve time that can be achieved by setting some of the available values to False.

**Using Thin Client**

Using thin client to hit the cuOpt endpoint, you would need to know the ip and port where the server is being run.

This provides few inherent options which can be a good starting point for new users,

In [ ]:
from cuopt_sh_client import CuOptServiceSelfHostClient

client = CuOptServiceSelfHostClient(ip=ip, port=port, use_https=False)

sol = client.get_optimized_routes(json_data)

print(json.dumps(sol, indent=4))

**Direct cuOpt API Endpoint**

Interacting directly with cuOpt API, this would be useful for customizing cuOpt into user workflow.

In [ ]:
import time

url = "http://" + ip + ":" + port + "/cuopt/"

# The request endpoint is asynchronous; it returns a request id
solver_response = requests.post(
    url + "request", json=json_data
).json()

# Now we poll the request id in a loop until we get a response
while True:
    reqId = solver_response["reqId"]
    solver_response = requests.get(url + f"solution/{reqId}").json()
    if "response" in solver_response or "error" in solver_response:
        break
    time.sleep(1)

print(json.dumps(solver_response, indent=4))

**Solver Routes**

Also shown here is a utility function for displaying the optimized result.

In [ ]:
def show_results(res):
    print("\n====================== Response ===========================\n")
    print("Solver status: ", res["status"])
    if res["status"] == 0:
        print("Cost         : ", res["solution_cost"])
        print("Vehicle count: ", res["num_vehicles"])
        for veh_id in res["vehicle_data"].keys():
            print("\nVehicle ID: ", veh_id)
            print("----------")
            print("Tasks assigned: ", res["vehicle_data"][veh_id]["task_id"])
            data = res["vehicle_data"][veh_id]
            routes_and_types = {key:data[key] for key in ["route", "type"]}
            print("Route: \n", routes_and_types)
    else:
        print("Error: ", res["error"])
    print("\n======================= End ===============================\n")
    
show_results(solver_response["response"]["solver_response"])